In [6]:
# Python 2/3 compatibility imports
from __future__ import print_function
from six.moves import input


import sys
import time
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg


try:
    from math import pi, tau, dist, fabs, cos
except:  # For Python 2 compatibility
    from math import pi, fabs, cos, sqrt

    tau = 2.0 * pi

    def dist(p, q):
        return sqrt(sum((p_i - q_i) ** 2.0 for p_i, q_i in zip(p, q)))


from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

from controller_manager_msgs.srv import SwitchController
from controller_manager_msgs.srv import SwitchControllerRequest

from franka_msgs.srv import SetJointImpedance
from franka_msgs.srv import SetJointImpedanceRequest

In [7]:

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("move_group_python_interface_tutorial", anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

group_name = "panda_arm"
move_group = moveit_commander.MoveGroupCommander(group_name)

group_name_hand = "panda_hand"
move_group_hand = moveit_commander.MoveGroupCommander(group_name_hand)

display_trajectory_publisher = rospy.Publisher(
"/move_group/display_planned_path",
moveit_msgs.msg.DisplayTrajectory,
queue_size=20,
)
planning_frame = move_group.get_planning_frame()
#print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
#print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
#print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
#print("============ Printing robot state")
#print(robot.get_current_state())

switch_controller = rospy.ServiceProxy('/controller_manager/switch_controller', SwitchController)

set_joint_impedance = rospy.ServiceProxy('/franka_control/set_joint_impedance', SetJointImpedance)
print("================== Initialization Complete")

================== Initialization Complete


In [28]:
import actionlib
from franka_gripper.msg import MoveAction, GraspAction, MoveGoal, GraspGoal
gripper_move_client = actionlib.SimpleActionClient('/franka_gripper/move', MoveAction)
gripper_grasp_client = actionlib.SimpleActionClient('/franka_gripper/grasp', GraspAction)

In [51]:
# fully open

goal = MoveGoal()
goal.width = 0.08
goal.speed = 0.1
gripper_move_client.send_goal_and_wait(goal)
gripper_move_client.get_result()

success: True
error: ''

In [49]:
# close on object
def grab_object():
    
    goal = GraspGoal()
    goal.width = 0.018
    goal.epsilon.inner = 0.005
    goal.epsilon.outer = 0.005
    goal.speed = 0.1
    goal.force = 10.
    gripper_grasp_client.send_goal_and_wait(goal)
    gripper_grasp_client.get_result()
    print("================= Grab the Object")

In [53]:
grab_object()

In [20]:
def recover():
    
    !rostopic pub -1 /franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"
    print("================== Recovered from Error")
    

In [ ]:
def take_pic(time):
    from sensor_msgs.msg import Image
    img_rgb = rospy.wait_for_message("/camera/color/image_raw",Image)
    img_depth = rospy.wait_for_message("/camera/depth/image_rect_raw",Image)

    from cv_bridge import CvBridge
    bridge = CvBridge()
    cv_image_rgb = bridge.imgmsg_to_cv2(img_rgb, desired_encoding='passthrough')
    cv_image_d = bridge.imgmsg_to_cv2(img_depth, desired_encoding='passthrough')

    import matplotlib.pylab as plt
    from matplotlib.pyplot import figure

    figure(figsize=(20,20), dpi=80)
    plt.imshow(cv_image_rgb)

In [4]:
def set_vel(vel1=0.3, acc1=0.3, vel2=0.02, acc2=0.02):
    move_group.set_max_velocity_scaling_factor(vel1)
    move_group.set_max_acceleration_scaling_factor(acc1)
    move_group_hand.set_max_velocity_scaling_factor(vel2)
    move_group_hand.set_max_acceleration_scaling_factor(acc2)
    
    print("================== Joints speed: " + str(vel1))
    print("================== Joints acceleration: " + str(acc1))
    print("================== Hands speed: " + str(vel2))
    print("================== Hands acceleration: " + str(acc2))

In [5]:
def reset():
    
    move_group.set_max_velocity_scaling_factor(0.1)
    move_group.set_max_acceleration_scaling_factor(0.1)
    
    midgrip(0.02)
    
    waypoints = []

    wpose = move_group.get_current_pose().pose
    wpose.position.x += 0  
    wpose.position.y += 0
    wpose.position.z += 0.1
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold

    # Note: We are just planning, not asking move_group to actually move the robot yet:

    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.1,
                                 acceleration_scaling_factor=0.1)
    
    #display_trajectory = moveit_msgs.msg.DisplayTrajectory()
    #display_trajectory.trajectory_start = robot.get_current_state()
    #display_trajectory.trajectory.append(new_plan)
    # Publish
    #display_trajectory_publisher.publish(display_trajectory)
    
    move_group.execute(new_plan, wait=True)
  
    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = 0 * pi /180
    joint_goal[1] = -48 * pi / 180
    joint_goal[2] = 0 * pi / 180
    joint_goal[3] = -153 * pi / 180
    joint_goal[4] = 0
    joint_goal[5] = 106 * pi / 180
    joint_goal[6] = 44 * pi / 180

    move_group.go(joint_goal, wait=True)

    move_group.stop()
    
    print("================== Reset to home position")

In [6]:
def go_to_button():
    
    move_group.set_max_velocity_scaling_factor(0.1)
    move_group.set_max_acceleration_scaling_factor(0.1)
    
    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = -65 * pi /180
    joint_goal[1] = -78 * pi / 180
    joint_goal[2] = 91 * pi / 180
    joint_goal[3] = -167 * pi / 180
    joint_goal[4] = 88 * pi / 180
    joint_goal[5] = 103 * pi / 180
    joint_goal[6] = -6 * pi / 180
    
    move_group.go(joint_goal, wait=True)
    
    move_group.stop()
    
    print("================== Arm at button now")

In [7]:
def grip():
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = 0
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()
    
    print("================== Hands gripped")
    

def ungrip():
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = 0.039
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()
    
    print("================== Hands ungripped")

def midgrip(dist):
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = dist
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()
    
    print("================== Hands gripped to " + str(dist))

In [8]:
def move_cart(x,y,z):
    waypoints = []

    wpose = move_group.get_current_pose().pose
    wpose.position.x -= x  
    wpose.position.y -= y  
    wpose.position.z -= z  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold

    # Note: We are just planning, not asking move_group to actually move the robot yet:

    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.1,
                                 acceleration_scaling_factor=0.1)
    
    #display_trajectory = moveit_msgs.msg.DisplayTrajectory()
    #display_trajectory.trajectory_start = robot.get_current_state()
    #display_trajectory.trajectory.append(new_plan)
    # Publish
    #display_trajectory_publisher.publish(display_trajectory)
    
    move_group.execute(new_plan, wait=True)

In [21]:
recover()

publishing and latching message for 3.0 seconds
================== Recovered from Error


In [50]:
move_cart(0.00,0.00,-0.005)

In [9]:
def push_button():

    ungrip()
    print("================== Adjusting to soft stiffness")
    
    soft()
    grip()
    print("================== Stiffness adjustion complete")
    
    
    waypoints = []

    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    
    wpose.position.z -= 0.016
    waypoints.append(copy.deepcopy(wpose))

    wpose.position.z += 0.05  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold



    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    move_group.execute(new_plan, wait=True)
    
    print("================== Adjusting to rigid stiffness")
    rigid()
    
    print("================== Stiffness adjustion complete")



In [10]:
def stop_controllers():
    req = SwitchControllerRequest()
    req.stop_controllers = ['franka_state_controller','position_joint_trajectory_controller']
    switch_controller(req)
    
def start_controllers():
    req = SwitchControllerRequest()
    req.start_controllers = ['franka_state_controller','position_joint_trajectory_controller']
    switch_controller(req)

def set_joint_stiffness(stiffness):
    req = SetJointImpedanceRequest()
    req.joint_stiffness = stiffness
    set_joint_impedance(req)
    
def rigid():
    stop_controllers()
    stiffness = [10000.,10000.,10000., 10000.,10000.,10000.,10000]
    set_joint_stiffness(stiffness)
    start_controllers()

def soft():
    stop_controllers()
    stiffness = [100.,25.,100., 100.,100.,100.,100]
    set_joint_stiffness(stiffness)
    start_controllers()

In [11]:
def go_to_door():
    
    move_group.set_max_velocity_scaling_factor(0.2)
    move_group.set_max_acceleration_scaling_factor(0.2)

    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = -40 * pi /180
    joint_goal[1] = -41 * pi / 180
    joint_goal[2] = +81 * pi / 180
    joint_goal[3] = -164 * pi / 180
    joint_goal[4] = +65 * pi / 180
    joint_goal[5] = +135 * pi / 180
    joint_goal[6] = +29 * pi / 180
    
    move_group.go(joint_goal, wait=True)
    move_group.stop()
    
    print("================== Arm at door")


In [12]:
def grab_door():

    

    
    midgrip(0.02)
    print("================== Ready to grip")
    
    waypoints = []

    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    wpose.position.z -= 0.028
    waypoints.append(copy.deepcopy(wpose))
    
    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold

    

    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    print("================== Moving down")
    
    move_group.execute(new_plan, wait=True)
    
    print("================== Adjusting stiffness to soft")

   
    soft()
    
    print("================== Stiffness adjustion complete")

    midgrip(0.01)
    
    print("================== Hands gripped")
    
    waypoints = []
    
    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    wpose.position.z += 0.03  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold



    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    print("================== Moving up stage 1")
    
    move_group.execute(new_plan, wait=True)
    
    #########################
    set_vel(vel1=0.01, acc1=0.01, vel2=0.02, acc2=0.02)
    
    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = -0.3204
    joint_goal[1] = -1.0127
    joint_goal[2] = +1.6042
    joint_goal[3] = -3.0560
    joint_goal[4] = +1.6204
    joint_goal[5] = +2.5304
    joint_goal[6] = +0.5540
    
    print("================== Moving up stage 2")
    
    move_group.go(joint_goal, wait=True)
    move_group.stop()
    
    
    ungrip()
    
    print("================== Adjusting stiffness to rigid")
    
    
    rigid()
    
    print("================== Stiffness adjustion complete")


In [13]:
#initialization()
set_vel()

================== Joints speed: 0.3
================== Joints acceleration: 0.3
================== Hands speed: 0.02
================== Hands acceleration: 0.02


In [16]:
recover()

publishing and latching message for 3.0 seconds
================== Recovered from Error


In [19]:
reset()

================== Hands gripped to 0.02
================== Reset to home position


In [18]:
go_to_button()

================== Arm at button now


In [30]:
move_group.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1681332084
    nsecs:  97629070
  frame_id: "panda_link0"
pose: 
  position: 
    x: 0.297374955619
    y: 0.000183468011033
    z: 0.215219782794
  orientation: 
    x: 0.921607821385
    y: -0.388000586845
    z: 0.00735878053458
    w: 0.00635739872174

In [22]:
push_button()

================== Hands ungripped
================== Adjusting to soft stiffness
================== Hands gripped
================== Stiffness adjustion complete
z 0.477459718861
================== Adjusting to rigid stiffness
================== Stiffness adjustion complete


In [31]:
go_to_door()

================== Arm at door


In [32]:
move_group.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1681332092
    nsecs: 531099081
  frame_id: "panda_link0"
pose: 
  position: 
    x: 0.294594800611
    y: 0.142231506462
    z: 0.231712613918
  orientation: 
    x: -0.917385701368
    y: 0.397934537487
    z: -0.00670234692489
    w: 0.0025801832939

In [24]:
grab_door()

================== Hands gripped to 0.02
================== Ready to grip
z 0.23178706058
================== Moving down
================== Adjusting stiffness to soft
================== Stiffness adjustion complete
================== Hands gripped to 0.01
================== Hands gripped
z 0.203500115312
================== Moving up stage 1
================== Joints speed: 0.01
================== Joints acceleration: 0.01
================== Hands speed: 0.02
================== Hands acceleration: 0.02
================== Moving up stage 2
================== Hands ungripped
================== Adjusting stiffness to rigid
================== Stiffness adjustion complete


In [120]:
move_group.get_current_joint_values()

[-0.9841372454375551,
 -1.756996111032132,
 1.7448990276905547,
 -2.9297922391188913,
 1.803315358771218,
 1.4854067503167867,
 0.02055651079769826]

In [4]:
move_group.get_current_joint_values()

[-1.1331192768665788,
 -1.103127744380191,
 1.607523641636495,
 -2.8604405216132456,
 1.415821283561884,
 2.0169349999957564,
 -0.0020888187022506293]

In [113]:
ungrip()

================== Hands ungripped


In [109]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] = -0.3204
joint_goal[1] = -1.0127
joint_goal[2] = +1.6042
joint_goal[3] = -3.0560
joint_goal[4] = +1.6204
joint_goal[5] = +2.5304
joint_goal[6] = +0.5540

print("================== Moving up stage 2")

move_group.go(joint_goal, wait=True)
move_group.stop()


================== Moving up stage 2


In [5]:
move_group.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1681331297
    nsecs: 630969047
  frame_id: "panda_link0"
pose: 
  position: 
    x: 0.309426437512
    y: 0.0280141023692
    z: 0.202691574409
  orientation: 
    x: 0.921844726067
    y: -0.386848185563
    z: -0.0145063418546
    w: 0.0184485336712

In [40]:
import eigenpy
import numpy

In [37]:
help(eigenpy.AngleAxis)

Help on class AngleAxis in module eigenpy.eigenpy_pywrap:

class AngleAxis(Boost.Python.instance)
 |  AngleAxis representation of a rotation.
 |  
 |  Method resolution order:
 |      AngleAxis
 |      Boost.Python.instance
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(...)
 |      __eq__( (AngleAxis)arg1, (AngleAxis)arg2) -> bool :
 |      
 |          C++ signature :
 |              bool __eq__(Eigen::AngleAxis<double>,Eigen::AngleAxis<double>)
 |  
 |  __init__(...)
 |      __init__( (object)self) -> None :
 |          Default constructor
 |      
 |          C++ signature :
 |              void __init__(_object*)
 |      
 |      __init__( (object)self, (float)angle, (numpy.ndarray)axis) -> None :
 |          Initialize from angle and axis.
 |      
 |          C++ signature :
 |              void __init__(_object*,double,Eigen::Matrix<double, 3, 1, 0, 3, 1>)
 |      
 |      __init__( (object)self, (numpy.ndarray)R) -> None :
 |          Initialize from

In [48]:
axis = numpy.array([[0.,0.,1.]])
aa = eigenpy.AngleAxis(0.1,axis.T)

In [49]:
aa.matrix()

array([[ 0.99500417, -0.09983342,  0.        ],
       [ 0.09983342,  0.99500417,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [50]:
help(eigenpy.Quaternion)

Help on class Quaternion in module eigenpy.eigenpy_pywrap:

class Quaternion(Boost.Python.instance)
 |  Quaternion representing rotation.
 |  
 |  Supported operations ('q is a Quaternion, 'v' is a Vector3): 'q*q' (rotation composition), 'q*=q', 'q*v' (rotating 'v' by 'q'), 'q==q', 'q!=q', 'q[0..3]'.
 |  
 |  Method resolution order:
 |      Quaternion
 |      Boost.Python.instance
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __abs__(...)
 |      __abs__( (Quaternion)arg1) -> float :
 |      
 |          C++ signature :
 |              double __abs__(Eigen::Quaternion<double, 0> {lvalue})
 |  
 |  __eq__(...)
 |      __eq__( (Quaternion)arg1, (Quaternion)arg2) -> bool :
 |      
 |          C++ signature :
 |              bool __eq__(Eigen::Quaternion<double, 0>,Eigen::Quaternion<double, 0>)
 |  
 |  __getitem__(...)
 |      __getitem__( (Quaternion)arg1, (int)arg2) -> float :
 |      
 |          C++ signature :
 |              double __getitem__(Eigen::Quaternio

In [70]:
#(float)w, (float)x, (float)y, (float)z) -> object 
p = move_group.get_current_pose()
q = eigenpy.Quaternion(p.pose.orientation.w,p.pose.orientation.x,p.pose.orientation.y,p.pose.orientation.z)
print(q)

(x,y,z,w) =   -0.927195     0.37448 -0.00803636  0.00326439



In [71]:
q.toRotationMatrix()

array([[  7.19400609e-01,  -6.94378737e-01,   1.73474284e-02],
       [ -6.94483672e-01,  -7.19508463e-01,   3.45470881e-05],
       [  1.24576328e-02,  -1.20723590e-02,  -9.99849521e-01]])

In [72]:
numpy.set_printoptions(precision=3)

In [73]:
T = numpy.hstack([
    numpy.vstack([q.toRotationMatrix(),numpy.array([0.,0.,0.])]),
    numpy.array([[p.pose.position.x,p.pose.position.y,p.pose.position.z,1]]).T])
T

array([[  7.194e-01,  -6.944e-01,   1.735e-02,   3.025e-01],
       [ -6.945e-01,  -7.195e-01,   3.455e-05,   5.911e-05],
       [  1.246e-02,  -1.207e-02,  -9.998e-01,   4.806e-01],
       [  0.000e+00,   0.000e+00,   0.000e+00,   1.000e+00]])

In [74]:
point = numpy.array([[0.,0.,0.1,1.]]).T
numpy.matmul(T, point)

array([[  3.042e-01],
       [  6.256e-05],
       [  3.806e-01],
       [  1.000e+00]])

In [132]:
move_group.set_end_effector_link("panda_hand_tcp")
import eigenpy
import numpy
p = move_group.get_current_pose()
print(p)
q = eigenpy.Quaternion(p.pose.orientation.w,p.pose.orientation.x,p.pose.orientation.y,p.pose.orientation.z)
print("q",q)
print("q_rev\n",eigenpy.Quaternion(q.toRotationMatrix()).inverse())
R_T_G = numpy.hstack([
    numpy.vstack([q.toRotationMatrix(),numpy.array([0.,0.,0.])]),
    numpy.array([[p.pose.position.x,p.pose.position.y,p.pose.position.z,1]]).T])
print("R_T_G\n",R_T_G)
axis = numpy.array([[0.,0.,1.]])
aa = eigenpy.AngleAxis(0.0,axis.T)
T_offset = numpy.hstack([
    numpy.vstack([aa.toRotationMatrix(),numpy.array([0.,0.,0.])]),
    numpy.array([[0.1,0.,0.,1]]).T])
print("T_offset\n",T_offset)
R_T_G_new = numpy.matmul(R_T_G,T_offset)
print("R_T_G_new\n",R_T_G_new)
print(R_T_G_new[0:3,0:3])
q_new = eigenpy.Quaternion(R_T_G_new[0:3,0:3]).inverse() # WTF why inverse!
print("q_new",q_new)

header: 
  seq: 0
  stamp: 
    secs: 1681336968
    nsecs: 903101921
  frame_id: "panda_link0"
pose: 
  position: 
    x: 0.304263494357
    y: 9.8663190748e-08
    z: 0.377169635504
  orientation: 
    x: 0.9999233286
    y: 0.00881990980121
    z: 0.00869118287495
    w: 9.72281138225e-05
q (x,y,z,w) =    0.999923  0.00881991  0.00869118 9.72281e-05

q_rev
 (x,y,z,w) =   -0.999923 -0.00881991 -0.00869118 9.72281e-05

R_T_G
 [[  9.997e-01   1.764e-02   1.738e-02   3.043e-01]
 [  1.764e-02  -9.998e-01  -4.113e-05   9.866e-08]
 [  1.738e-02   3.478e-04  -9.998e-01   3.772e-01]
 [  0.000e+00   0.000e+00   0.000e+00   1.000e+00]]
T_offset
 [[ 1.   0.   0.   0.1]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.   1. ]]
R_T_G_new
 [[  9.997e-01   1.764e-02   1.738e-02   4.042e-01]
 [  1.764e-02  -9.998e-01  -4.113e-05   1.764e-03]
 [  1.738e-02   3.478e-04  -9.998e-01   3.789e-01]
 [  0.000e+00   0.000e+00   0.000e+00   1.000e+00]]
[[  9.997e-01   1.764e-02   1.738e-02]
 [  1.

In [133]:
move_group.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1681336969
    nsecs: 402893066
  frame_id: "panda_link0"
pose: 
  position: 
    x: 0.304262988834
    y: 1.12171624127e-06
    z: 0.377168838321
  orientation: 
    x: 0.999923311505
    y: 0.00882339519645
    z: 0.00868960869783
    w: 9.75022088537e-05

In [134]:
waypoints = []

wpose = move_group.get_current_pose().pose
#print(wpose)
#print(wpose.position)
wpose.position.x = R_T_G_new[0,3]
wpose.position.y = R_T_G_new[1,3]
wpose.position.z = R_T_G_new[2,3]
#print(wpose.position)

#print(wpose.orientation)

wpose.orientation.x = q_new.x
wpose.orientation.y = q_new.y
wpose.orientation.z = q_new.z
wpose.orientation.w = q_new.w
#print(wpose.orientation)
waypoints.append(copy.deepcopy(wpose))
print(waypoints)
(plan, fraction) = move_group.compute_cartesian_path(
    waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
)  # jump_threshold

# Note: We are just planning, not asking move_group to actually move the robot yet:

new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                             plan,velocity_scaling_factor=0.1,
                             acceleration_scaling_factor=0.1)

display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = robot.get_current_state()
display_trajectory.trajectory.append(new_plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

move_group.execute(new_plan, wait=True)

[position: 
  x: 0.404232828863
  y: 0.00176411438193
  z: 0.378907567298
orientation: 
  x: -0.9999233286
  y: -0.00881990980121
  z: -0.00869118287495
  w: 9.72281138225e-05]


True

In [123]:
reset()

================== Hands gripped to 0.02
================== Reset to home position


Help on method compute_cartesian_path in module moveit_commander.move_group:

compute_cartesian_path(self, waypoints, eef_step, jump_threshold, avoid_collisions=True, path_constraints=None) method of moveit_commander.move_group.MoveGroupCommander instance
    Compute a sequence of waypoints that make the end-effector move in straight line segments that follow the poses specified as waypoints. Configurations are computed for every eef_step meters; The jump_threshold specifies the maximum distance in configuration space between consecutive points in the resultingpath; Kinematic constraints for the path given by path_constraints will be met for every point along the trajectory, if they are not met, a partial solution will be returned. The return value is a tuple: a fraction of how much of the path was followed, the actual RobotTrajectory.

